In [1]:
!pip install catboost scikit-learn autogluon

# Autogluon Model

In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from autogluon.tabular import TabularPredictor
import autogluon

## Define constans and functions

In [3]:
locations = ["A", "B", "C"]
features_order = []

LAGGED_COLUMNS_TO_KEEP = [
    # 'direct_rad:W_lag_1h', 
    'direct_rad:W_lag_forward_1h', 
    # 'clear_sky_rad:W_lag_1h', 
    'clear_sky_rad:W_lag_forward_1h', 
    # 'diffuse_rad:W_lag_1h', 
    'diffuse_rad:W_lag_forward_1h', 
    # 'direct_rad_1h:J_lag_1h', 
    'direct_rad_1h:J_lag_forward_1h', 
    # 'is_in_shadow:idx_lag_1h', 
    'is_in_shadow:idx_lag_forward_1h', 
    # 'clear_sky_energy_1h:J_lag_1h', 
    'clear_sky_energy_1h:J_lag_forward_1h', 
    # 'effective_cloud_cover:p_lag_1h', 
    'effective_cloud_cover:p_lag_forward_1h', 
    # 'visibility:m_lag_1h', 
    'visibility:m_lag_forward_1h', 
    # 'total_cloud_cover:p_lag_1h', 
    'total_cloud_cover:p_lag_forward_1h', 


    # 'direct_rad:W_lag_2h', 
    # 'direct_rad:W_lag_forward_2h', 
    # 'clear_sky_rad:W_lag_2h', 
    # 'clear_sky_rad:W_lag_forward_2h', 
    # 'diffuse_rad:W_lag_2h', 
    # 'diffuse_rad:W_lag_forward_2h', 
    # 'direct_rad_1h:J_lag_2h', 
    # 'direct_rad_1h:J_lag_forward_2h', 
    # 'is_in_shadow:idx_lag_2h', 
    # 'is_in_shadow:idx_lag_forward_2h', 
    # 'clear_sky_energy_1h:J_lag_2h', 
    # 'clear_sky_energy_1h:J_lag_forward_2h', 
    # 'effective_cloud_cover:p_lag_2h', 
    # 'effective_cloud_cover:p_lag_forward_2h', 
    # 'visibility:m_lag_2h', 
    # 'visibility:m_lag_forward_2h', 
    # 'total_cloud_cover:p_lag_2h', 
    # 'total_cloud_cover:p_lag_forward_2h', 

    # 'direct_rad:W_lag_3h', 
    # 'direct_rad:W_lag_forward_3h', 
    # 'clear_sky_rad:W_lag_3h', 
    # 'clear_sky_rad:W_lag_forward_3h', 
    # 'diffuse_rad:W_lag_3h', 
    # 'diffuse_rad:W_lag_forward_3h', 
    # 'direct_rad_1h:J_lag_3h', 
    # 'direct_rad_1h:J_lag_forward_3h', 
    # 'is_in_shadow:idx_lag_3h', 
    # 'is_in_shadow:idx_lag_forward_3h', 
    # 'clear_sky_energy_1h:J_lag_3h', 
    # 'clear_sky_energy_1h:J_lag_forward_3h', 
    # 'effective_cloud_cover:p_lag_3h', 
    # 'effective_cloud_cover:p_lag_forward_3h', 
    # 'visibility:m_lag_3h', 
    # 'visibility:m_lag_forward_3h', 
    # 'total_cloud_cover:p_lag_3h', 
    # 'total_cloud_cover:p_lag_forward_3h'
]

CUSTOM_COLUMNS_TO_KEEP = [
    "hour_cos",
    "hour_sin",
    "month_sin",
    "month_cos",
    "day-of-year",
    "hours_since_forecast"
]

WEATHER_FEATURES = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
]


TEST_COLUMNS_TO_KEEP = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "diffuse_rad_1h:J",
    "is_day:idx",
    "sun_elevation:d",
    "ceiling_height_agl:m",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
    "air_density_2m:kgm3",
    "wind_speed_v_10m:ms",
    "dew_point_2m:K",
    "wind_speed_u_10m:ms",
    "t_1000hPa:K",
    "absolute_humidity_2m:gm3",
    "snow_water:kgm2",
    "relative_humidity_1000hPa:p",
    "fresh_snow_24h:cm",
    "cloud_base_agl:m",
    "fresh_snow_12h:cm",
    "snow_depth:cm",
    "dew_or_rime:idx",
    "fresh_snow_6h:cm",
    "super_cooled_liquid_water:kgm2",
    "fresh_snow_3h:cm",
    "rain_water:kgm2",
    "precip_type_5min:idx",
    "precip_5min:mm",
    "fresh_snow_1h:cm",
    "sun_azimuth:d",
    "msl_pressure:hPa",
    "pressure_100m:hPa",
    "pressure_50m:hPa",
    "sfc_pressure:hPa",
    "prob_rime:p",
    "wind_speed_10m:ms",
    # "elevation:m",
    # "snow_density:kgm3",
    # "snow_drift:idx",
    "snow_melt_10min:mm",
    "wind_speed_w_1000hPa:ms",
    "observed_or_estimated"
    # "location_A",
    # "location_B",
    # "location_C",
    # "date_calc",
] + CUSTOM_COLUMNS_TO_KEEP  +  LAGGED_COLUMNS_TO_KEEP

COLUMNS_TO_KEEP = TEST_COLUMNS_TO_KEEP + ["pv_measurement"]



def create_weather_lagged_features(df, weather_features):
    # Choose the weather features for which you want to create lagged versions
    for feature in weather_features:
        # Assuming hourly data, adjust the lags for your specific dataset
        # Creating lagged features for 1 hour, 1 day, and 1 week
        # df[f'{feature}_lag_1h'] = df[feature].shift(1)
        # df[f'{feature}_lag_2h'] = df[feature].shift(2)
        # df[f'{feature}_lag_3h'] = df[feature].shift(3)

        df[f'{feature}_lag_forward_1h'] = df[feature].shift(-1)
        # df[f'{feature}_lag_forward_2h'] = df[feature].shift(-2)
        # df[f'{feature}_lag_forward_3h'] = df[feature].shift(-3)
        # df[f'{feature}_lag_24h'] = df[feature].shift(24*4)
        # df[f'{feature}_lag_168h'] = df[feature].shift(24 * 7 * 4 * 365)
        # df[f'{feature}_front_lag_1h'] = df[feature].shift(-4)
        # df[f'{feature}_front_lag_24h'] = df[feature].shift(-24*4)


    # Handling edges by filling NaNs with appropriate values or dropping them
    # You may choose to fill with zeroes or interpolate, based on what makes more sense for your data
    # df.fillna(method='ffill', inplace=True)  # Forward fill  # Autogluon should handle this for us.
    # df.fillna(method='bfill', inplace=True)  # Backward fill  # Autogluon should handle this for us.
    
    return df


B_SCALE_VALUE = 6.3
C_SCALE_VALUE = 8.1

## Prepare data


In [4]:
def add_custom_fields(df):
     df['hour_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.hour / 24)
     df['hour_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.hour / 24)

     df['month_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.month / 12)
     df['month_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.month / 12)
     df['day-of-year'] = df['date_forecast'].dt.dayofyear
     return df

def add_calc_date(df_observed, df_estimated, df_test):
    # Function to calculate the difference in hours
    def calculate_hour_difference(row):
        diff = row['date_calc'] - row['date_forecast']
        return diff.total_seconds() / 3600  # Convert difference to hours

    # Apply the function to calculate the hour difference for df_estimated and df_test
    df_estimated['hours_since_forecast'] = df_estimated.apply(calculate_hour_difference, axis=1)
    df_test['hours_since_forecast'] = df_test.apply(calculate_hour_difference, axis=1)

    # Fill in zero for df_observed
    df_observed['hours_since_forecast'] = 0

    return df_observed, df_estimated, df_test


def remove_outliers(df):
    # Use a mask to filter out the rows where rolling std is zero but keep the rows where the value itself is zero
    # Because some places in the data, the pv-measurements are messed up and are repeating.
    mask = (df['pv_measurement'].rolling(2).std() == 0) & (df['pv_measurement'] != 0)
    df[mask] = np.NaN  # Put this to NaN and hope autoGluon Handles.
    return df

def resample_add_data(df, is_test_data):
    df = add_custom_fields(df)
    df.set_index('date_forecast', inplace=True)
    df = df.resample('1H').mean()
    
    # Remove empty dates if test data
    if is_test_data:
        non_nan_threshold = len(df.columns) // 2
        df.dropna(thresh=non_nan_threshold, inplace=True)

    # df.interpolate(method="linear", inplace=True)  # Autogluon should handle this for us.
    
    return df

def add_location_feature(X, location):
      # Treat location as a categorical feature by converting it to a category type
    X['location'] = location
    X['location'] = X['location'].astype(str)  # Convert to string if 'location' is not an int
    # X['dew_or_rime:idx'] = X['dew_or_rime:idx'].astype(str)
    # X['is_day:idx'] = X['is_day:idx'].astype(str)
    # X['is_in_shadow:idx'] = X['is_in_shadow:idx'].astype(str)
    # categorical_columns = ['location', 'dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx']

    # # Before filling NaN values, add 'missing' as a category for each categorical column.
    # for column in categorical_columns:
    #     X[column] = X[column].astype('category')  # Ensure the column is of type 'category'.
    #     if 'missing' not in X[column].cat.categories:
    #         X[column] = X[column].cat.add_categories(['missing'])  # Add 'missing' as a new category.
    #     X[column] = X[column].fillna('missing')
    # X['location'] = X['location'].astype('category')
    # X['dew_or_rime:idx'] = X['dew_or_rime:idx'].astype('category')
    # X['is_day:idx'] = X['is_day:idx'].astype('category')
    # X['is_in_shadow:idx'] = X['is_in_shadow:idx'].astype('category')
    
    return X

def add_scaling(X_test, X_training, location):
    global scalers
    continuous_columns = X_training.select_dtypes(include=['float32', 'int32']).columns
    if location not in scalers:
        scalers[location] = MinMaxScaler()
    X_training[continuous_columns] = scalers[location].fit_transform(X_training[continuous_columns])
    
    X_test[continuous_columns] = scalers[location].transform(X_test[continuous_columns])

    return X_test, X_training

# Skip this as we have hours since forecast as a feature.
# Deprecated as the concat is moved to main function.
def make_observed_and_estimated_category(df_observed, df_estimated, df_test):
     # Hot encode in wether observed or estimated
    df_observed['observed_or_estimated'] = 'observed'
    df_estimated['observed_or_estimated'] = 'estimated'
    df_test['observed_or_estimated'] = 'estimated'
    # Concatenate observed and estimated
    df_training = pd.concat([df_observed, df_estimated], axis=0).sort_values(by="date_forecast")
    df_training['observed_or_estimated'] = df_training['observed_or_estimated'].astype('category')
    df_test['observed_or_estimated'] = df_test['observed_or_estimated'].astype('category')

    return df_training, df_test

    
    

In [5]:

# Initialize a dictionary to hold the scalers for each location

scalers = {}


def prepare_data(location):
    # Load data
    scaling = False  # Set scaling to True to enable individual scaling for each location
    global scalers
    global scale_target 
    scale_target = False

    # Load training data
    df_observed = pd.read_parquet(f"data/{location}/X_train_observed.parquet")
    df_estimated = pd.read_parquet(f"data/{location}/X_train_estimated.parquet")
    df_target = pd.read_parquet(f"data/{location}/train_targets.parquet")
    # drop nan values in target data, pv measurement
    df_target.dropna(inplace=True)

   
    # Load test data
    df_test = pd.read_parquet(f"data/{location}/X_test_estimated.parquet")

   
    # Add calculated date
    df_observed, df_estimated, df_test = add_calc_date(df_observed, df_estimated, df_test)
    

    # Resample and add custom fields

    df_observed = resample_add_data(df_observed, False)
    df_estimated = resample_add_data(df_estimated, False)
    df_test = resample_add_data(df_test, True)

    df_training, df_test = make_observed_and_estimated_category(df_observed, df_estimated, df_test)
    

    # Autogluon should scale for us.
    if scale_target:
        if location == "B":
            df_target["pv_measurement"] = df_target["pv_measurement"] * B_SCALE_VALUE
        elif location == "C":
            df_target["pv_measurement"] = df_target["pv_measurement"] * C_SCALE_VALUE
    
    # Merge training with target data
    df_training = pd.merge(df_training, df_target, left_on="date_forecast", right_on="time", how="inner")
    
    # Create lagged features and remove outliers training
    df_training = create_weather_lagged_features(df_training, WEATHER_FEATURES)
    df_training = df_training[COLUMNS_TO_KEEP]
    df_training = remove_outliers(df_training)

    df_test = create_weather_lagged_features(df_test, WEATHER_FEATURES)
    df_test = df_test[TEST_COLUMNS_TO_KEEP]


    # Add categories
    df_training = add_location_feature(df_training, location)
    X_test = add_location_feature(df_test, location)
    
    # Add scaling
    if scaling:
        X_test, X_training = add_scaling(X_test, X_training, location)
    df_test.reset_index(inplace=True)
    df_test.drop(columns=["date_forecast"], inplace=True)
    # y_training = np.log1p(y_training)
    return df_training, X_test



# Use prepare_data function

combined_df_train = []
combined_df_test = []
combined_df_validation = []

for location in locations:
    # Prepare the training data
    X_training, X_test = prepare_data(location)

    df_training, df_test = prepare_data(location)
    # split df training at 
    X_training, X_validation = train_test_split(df_training[df_training["observed_or_estimated"] == "estimated"], test_size=1440, shuffle=True)
    X_training = pd.concat([X_training, df_training[df_training["observed_or_estimated"] == "observed"]])
    X_training = shuffle(X_training)
    
    combined_df_train.append(X_training)
    combined_df_validation.append(X_validation)

    combined_df_test.append(X_test)



/var/tmp/ipykernel_101452/2117605410.py:36: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/2117605410.py:36: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/2117605410.py:36: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/2117605410.py:36: FutureWarning: Th

# Define hyperparameters

In [6]:
# All of these hyperparameters have been found by experimenting with some standard parameters in AutoGluon, and then only using the best ones for each location to make the train time shorter
lgbmXT = {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}
customlgbm = {'extra_trees': True, 'feature_fraction': 0.8, 'learning_rate': 0.02, 'min_data_in_leaf': 3, 'num_leaves': 21, 'ag_args': {'name_suffix': 'custom2'}}

custom_fastai = {'bs': 1024, 'emb_drop': 0.6, 'epochs': 48, 'layers': [200, 100, 50], 'lr': 0.008, 'ps': 0.09, 'ag_args': {'name_suffix': '_custom'}}

hyperparameters_a = {
    'NN_TORCH': {},
    'GBM': [lgbmXT, customlgbm, 'GBMLarge'],
    'FASTAI': [custom_fastai],
    'CAT': {},
}
hyperparameters_b_c = {
    'NN_TORCH': {},
    'GBM': [customlgbm],
    'KNN': [{'weights': 'uniform'}],
    'FASTAI': [{}, custom_fastai],
    'CAT': {},
}


## Train

In [7]:
def train_model(dataset):
    # Define the path where the AutoGluon models will be saved
    # enumerate all the locations
    for index, location in enumerate(locations):
        save_path = f"autogluon_models/shortnotebook_model{location}_gluon"

        if location == "A":
            hyperparameters = hyperparameters_a
        else:
            hyperparameters = hyperparameters_b_c

        # Initialize the TabularPredictor object
        model = TabularPredictor(
            label="pv_measurement", path=save_path, eval_metric="mae"
        )
        model.fit(
            train_data=combined_df_train[index].dropna(subset=["pv_measurement"]),
            tuning_data=combined_df_validation[index].dropna(subset=["pv_measurement"]),
            presets="experimental_zeroshot_hpo_hybrid",
            use_bag_holdout=True,
            hyperparameters=hyperparameters,
            # Got from the example in the docs
            # time_limit=60*60*2, # It did not use up the time
            num_bag_folds=2,
            num_bag_sets=2,
            num_stack_levels=1,
        )


def evaluate_model(X_val, y_val, location, model=None):
    if model is None:
        # If no model is passed, we assume the model has been previously saved and needs to be loaded
        save_path = f"autogluon_models_location_{location}"
        # model = TabularPredictor.load(save_path)
        model = autogluon.multimodal(save_path)

    # Predictions are made on the non-transformed validation data
    y_pred = model.predict(X_val)

    # If y_val was transformed (e.g., log1p), then apply the inverse transformation
    # y_val = np.expm1(y_val)
    # y_pred = np.expm1(y_pred)

    # Calculate the MAE
    mae = mean_absolute_error(y_val, y_pred)
    print(f"Location {location}, Mean Absolute Error: {mae}")


# Train the model using all available training data and the initial validation set for early stopping
train_model(combined_df_train)

# Evaluate the model using the same validation set
# evaluate_model(combined_X_val, combined_Y_val, location, model)

Presets specified: ['experimental_zeroshot_hpo_hybrid']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=2, num_bag_sets=2
Beginning AutoGluon training ...
AutoGluon will save models to "autogluon_models/shortnotebook_modelA_gluon/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.197-1 (2023-09-29)
Disk Space Avail:   97.09 GB / 105.09 GB (92.4%)
Train Data Rows:    32643
Train Data Columns: 59
Tuning Data Rows:    1440
Tuning Data Columns: 59
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 644.11247, 1175.38916)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: 

# Feature importance

In [9]:
# model.leaderboard(combined_df_validation, silent=True)
# model.feature_importance(combined_df_validation, time_limit=120)

# predictior = TabularPredictor.load(f"autogluon_models/test_modelA", require_version_match=False)


# # Get the leaderboard
# print("leaderboard")
# leaderboard = predictior.leaderboard(silent=True)

# # Retrieve the best model's name
# best_model = leaderboard.iloc[0]['model']
# print("Predictor info:")
# predictior.info()

# # Retrieve hyperparameters of the best model
# best_model_info = predictior.get_model_info(model=best_model)
# best_hyperparameters = best_model_info['hyperparameters']

# # Optionally, save or print these hyperparameters
# print("Best Model Hyperparameters:", best_hyperparameters)

# Make predictions

In [10]:

# Assuming you have defined WEATHER_FEATURES, TEST_COLUMNS_TO_KEEP, and other functions previously

def make_predictions(df_test_pred, location):
    eval_model = TabularPredictor.load(f"autogluon_models/shortnotebook_model{location}_gluon", require_version_match=False)
    preds = eval_model.predict(df_test_pred)
    return preds





## Evaluate locally


In [11]:
import matplotlib.pyplot as plt


def evaluate_model_locally(location, scalers):
    # Load the test data
    target_df = pd.read_parquet(f"data/{location}/train_targets.parquet")
    
    # Make predictions
    # filter x_validate to only include values from location
    pred_dataset = combined_df_validation[combined_df_validation["location"] == location]
    pred_dataset.reset_index(inplace=True)

    preds = make_predictions(pred_dataset.drop("pv_measurement", axis=1), location)[-720:]
    target = target_df.tail(720)["pv_measurement"].to_numpy()
    
    differences = preds - target
    # Count predictions lower than the actual
    lower_predictions = (differences < 0) & (target != 0)
    # Count predictions higher than the actual
    higher_predictions = (differences > 0) & (target != 0)

    # Biggest misreads
    absolute_differences = abs(differences)
    max_diff_index = absolute_differences.argmax()  # Index of the biggest difference
    # max_diff_value = absolute_differences[max_diff_index]  # Value of the biggest difference
    print(f"Number of predictions that are a lower value than the actual, given that the actual is not 0: {lower_predictions.sum()}")
    print(f"Number of predictions that are larger than the target, given that the target is not 0: {higher_predictions.sum()}")
    # print(f"The biggest misread is at index {max_diff_index} with a difference of {max_diff_value}")
    
    index = target_df.index[-720:]
    print(f'location: {location}')
    # Plotting
    plt.figure(figsize=(60,6))
    plt.plot(index, target, label="Target")
    plt.plot(index, preds, label="Predictions")
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Target vs Predictions')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()

for loc in locations:
    # evaluate_model_locally(loc, scalers)
    pass


## Submit to csv 1

In [12]:
df_submission = pd.read_csv("data/test.csv")

for index, location in enumerate(locations): 
    preds = make_predictions(combined_df_test[index], location)

    # Assign the predictions to df_submission for the current location
    mask = df_submission["location"] == location
    # Add a check to make sure the lengths match
    if len(preds) != mask.sum():
        raise ValueError(f"Mismatch in length of predictions and submission entries for location {location}.")

    df_submission.loc[mask, "prediction"] = preds.to_numpy()


# Save the results to a new submission file
df_submission[["id", "prediction"]].to_csv("Short-notebook-Autogluon-erik.csv", index=False)

# Catboost Model

In [13]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


## Define constans and functions

In [14]:
locations = ["A", "B", "C"]
features_order = []
A_B_ratio = 6.73
A_C_ratio = 8.17

LAGGED_COLUMNS_TO_KEEP = [
    'direct_rad:W_lag_1h', 
    'direct_rad:W_lag_forward_1h', 
    'clear_sky_rad:W_lag_1h', 
    'clear_sky_rad:W_lag_forward_1h', 
    'diffuse_rad:W_lag_1h', 
    'diffuse_rad:W_lag_forward_1h', 
    'direct_rad_1h:J_lag_1h', 
    'direct_rad_1h:J_lag_forward_1h', 
    'is_in_shadow:idx_lag_1h', 
    'is_in_shadow:idx_lag_forward_1h', 
    'clear_sky_energy_1h:J_lag_1h', 
    'clear_sky_energy_1h:J_lag_forward_1h', 
    'effective_cloud_cover:p_lag_1h', 
    'effective_cloud_cover:p_lag_forward_1h', 
    'visibility:m_lag_1h', 
    'visibility:m_lag_forward_1h', 
    'total_cloud_cover:p_lag_1h', 
    'total_cloud_cover:p_lag_forward_1h', 


    # 'direct_rad:W_lag_2h', 
    # 'direct_rad:W_lag_forward_2h', 
    # 'clear_sky_rad:W_lag_2h', 
    # 'clear_sky_rad:W_lag_forward_2h', 
    # 'diffuse_rad:W_lag_2h', 
    # 'diffuse_rad:W_lag_forward_2h', 
    # 'direct_rad_1h:J_lag_2h', 
    # 'direct_rad_1h:J_lag_forward_2h', 
    # 'is_in_shadow:idx_lag_2h', 
    # 'is_in_shadow:idx_lag_forward_2h', 
    # 'clear_sky_energy_1h:J_lag_2h', 
    # 'clear_sky_energy_1h:J_lag_forward_2h', 
    # 'effective_cloud_cover:p_lag_2h', 
    # 'effective_cloud_cover:p_lag_forward_2h', 
    # 'visibility:m_lag_2h', 
    # 'visibility:m_lag_forward_2h', 
    # 'total_cloud_cover:p_lag_2h', 
    # 'total_cloud_cover:p_lag_forward_2h', 

    # 'direct_rad:W_lag_3h', 
    # 'direct_rad:W_lag_forward_3h', 
    # 'clear_sky_rad:W_lag_3h', 
    # 'clear_sky_rad:W_lag_forward_3h', 
    # 'diffuse_rad:W_lag_3h', 
    # 'diffuse_rad:W_lag_forward_3h', 
    # 'direct_rad_1h:J_lag_3h', 
    # 'direct_rad_1h:J_lag_forward_3h', 
    # 'is_in_shadow:idx_lag_3h', 
    # 'is_in_shadow:idx_lag_forward_3h', 
    # 'clear_sky_energy_1h:J_lag_3h', 
    # 'clear_sky_energy_1h:J_lag_forward_3h', 
    # 'effective_cloud_cover:p_lag_3h', 
    # 'effective_cloud_cover:p_lag_forward_3h', 
    # 'visibility:m_lag_3h', 
    # 'visibility:m_lag_forward_3h', 
    # 'total_cloud_cover:p_lag_3h', 
    # 'total_cloud_cover:p_lag_forward_3h'
]

CUSTOM_COLUMNS_TO_KEEP = [
    "hour_cos",
    "hour_sin",
    "month_sin",
    "month_cos",
    "day-of-year",
    # "hours_since_forecast",
    # "sun_azimuth_cos",
    # "sun_azimuth_sin",
]

WEATHER_FEATURES = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
]


TEST_COLUMNS_TO_KEEP = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "diffuse_rad_1h:J",
    "is_day:idx",
    "sun_elevation:d",
    "ceiling_height_agl:m",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
    "air_density_2m:kgm3",
    "wind_speed_v_10m:ms",
    "dew_point_2m:K",
    "wind_speed_u_10m:ms",
    "t_1000hPa:K",
    "absolute_humidity_2m:gm3",
    "snow_water:kgm2",
    "relative_humidity_1000hPa:p",
    "fresh_snow_24h:cm",
    "cloud_base_agl:m",
    "fresh_snow_12h:cm",
    "snow_depth:cm",
    "dew_or_rime:idx",
    "fresh_snow_6h:cm",
    "super_cooled_liquid_water:kgm2",
    "fresh_snow_3h:cm",
    "rain_water:kgm2",
    "precip_type_5min:idx",
    "precip_5min:mm",
    "fresh_snow_1h:cm",
    "sun_azimuth:d",
    "msl_pressure:hPa",
    "pressure_100m:hPa",
    "pressure_50m:hPa",
    "sfc_pressure:hPa",
    "prob_rime:p",
    "wind_speed_10m:ms",
    "elevation:m",
    # "snow_density:kgm3",
    "snow_drift:idx",
    "snow_melt_10min:mm",
    "wind_speed_w_1000hPa:ms",
    "observed_or_estimated",
    # "location_A",
    # "location_B",
    # "location_C",
    # "date_calc",
] + CUSTOM_COLUMNS_TO_KEEP  +  LAGGED_COLUMNS_TO_KEEP

COLUMNS_TO_KEEP = TEST_COLUMNS_TO_KEEP + ["pv_measurement"]



def create_weather_lagged_features(df, weather_features):
    # Choose the weather features for which you want to create lagged versions
    for feature in weather_features:
        # Assuming hourly data, adjust the lags for your specific dataset
        # Creating lagged features for 1 hour, 1 day, and 1 week
        df[f'{feature}_lag_1h'] = df[feature].shift(1)
        # df[f'{feature}_lag_2h'] = df[feature].shift(2)
        # df[f'{feature}_lag_3h'] = df[feature].shift(3)

        df[f'{feature}_lag_forward_1h'] = df[feature].shift(-1)
        # df[f'{feature}_lag_forward_2h'] = df[feature].shift(-2)
        # df[f'{feature}_lag_forward_3h'] = df[feature].shift(-3)
        # df[f'{feature}_lag_24h'] = df[feature].shift(24*4)
        # df[f'{feature}_lag_168h'] = df[feature].shift(24 * 7 * 4 * 365)
        # df[f'{feature}_front_lag_1h'] = df[feature].shift(-4)
        # df[f'{feature}_front_lag_24h'] = df[feature].shift(-24*4)


    # Handling edges by filling NaNs with appropriate values or dropping them
    # You may choose to fill with zeroes or interpolate, based on what makes more sense for your data
    df.fillna(method='ffill', inplace=True)  # Forward fill
    df.fillna(method='bfill', inplace=True)  # Backward fill
    
    return df


def create_lagged_features(df, column_name='pv_measurement'):
    # Assuming 'date_forecast' is the datetime column used for sorting

    df[f'{column_name}_prev_month'] = df[column_name].shift(24*7) # previous week

    # For yearly lag, you would need to calculate the number of observations per year
    # If the data is not consistent (leap years, etc.), you may need a more complex method
    # Here's a simple version assuming 365 days a year:
    df[f'{column_name}_prev_year'] = df[column_name].shift(24*365) # previous year
    df[f'{column_name}_2years_ago'] = df[column_name].shift(24*365*2) # next year

    # Handling edges by filling NaNs with appropriate values or dropping them
    df.fillna(method='ffill', inplace=True)  # Forward fill
    df.fillna(method='bfill', inplace=True)  # Backward fill

    return df


## Prepare data


In [15]:
def add_custom_fields(df):
    df['hour_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.hour / 24)

    df['month_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.month / 12)
    df['day-of-year'] = df['date_forecast'].dt.dayofyear

    # df['sun_azimuth_cos'] = np.cos(np.radians(df['sun_azimuth:d']))
    # df['sun_azimuth_sin'] = np.sin(np.radians(df['sun_azimuth:d']))
    # df.drop(columns=['sun_azimuth:d'], inplace=True)
   
    return df

def add_calc_date(df_observed, df_estimated, df_test):
    # Function to calculate the difference in hours
    def calculate_hour_difference(row):
        diff = row['date_calc'] - row['date_forecast']
        return diff.total_seconds() / 3600  # Convert difference to hours

    # Apply the function to calculate the hour difference for df_estimated and df_test
    df_estimated['hours_since_forecast'] = df_estimated.apply(calculate_hour_difference, axis=1)
    df_test['hours_since_forecast'] = df_test.apply(calculate_hour_difference, axis=1)

    # Fill in zero for df_observed
    df_observed['hours_since_forecast'] = 0

    return df_observed, df_estimated, df_test


def remove_outliers(df):
    # Use a mask to filter out the rows where rolling std is zero but keep the rows where the value itself is zero
    mask = (df['pv_measurement'].rolling(5).std() == 0) & (df['pv_measurement'] != 0)
    df = df[~mask]
    return df

def resample_add_data(df, is_test_data):
    df = add_custom_fields(df)
    df.set_index('date_forecast', inplace=True)
    df = df.resample('1H').mean()
    
    # Remove empty dates if test data
    if is_test_data:
        non_nan_threshold = len(df.columns) // 2  
        df.dropna(thresh=non_nan_threshold, inplace=True)
    df.interpolate(method="linear", inplace=True)
    
    return df

def add_location_feature(X, location):
      # Treat location as a categorical feature by converting it to a category type
    
    # bins = [-90, 0, 45, 90]
    # labels = ['Low', 'Medium', 'High']
    # X['sun_elevation:d'] = pd.cut(X['sun_elevation:d'], bins=bins, labels=labels, include_lowest=True)
    X['location'] = location
    X['location'] = X['location'].astype(str)  # Convert to string if 'location' is not an int
    X['dew_or_rime:idx'] = X['dew_or_rime:idx'].astype(str)
    X['is_day:idx'] = X['is_day:idx'].astype(str)
    X['is_in_shadow:idx'] = X['is_in_shadow:idx'].astype(str)
    categorical_columns = ['location', 'dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx']

    # Before filling NaN values, add 'missing' as a category for each categorical column.
    for column in categorical_columns:
        X[column] = X[column].astype('category')  # Ensure the column is of type 'category'.
        if 'missing' not in X[column].cat.categories:
            X[column] = X[column].cat.add_categories(['missing'])  # Add 'missing' as a new category.
        X[column] = X[column].fillna('missing')
    X['location'] = X['location'].astype('category')
    X['dew_or_rime:idx'] = X['dew_or_rime:idx'].astype('category')
    X['is_day:idx'] = X['is_day:idx'].astype('category')
    X['is_in_shadow:idx'] = X['is_in_shadow:idx'].astype('category')
    # X['sun_elevation:d'] = X['sun_elevation:d'].astype('category')
    
    return X

def add_scaling(X_test, X_training, location):
    global scalers
    continuous_columns = X_training.select_dtypes(include=['float32', 'int32']).columns
    if location not in scalers:
        scalers[location] = MinMaxScaler()
    X_training[continuous_columns] = scalers[location].fit_transform(X_training[continuous_columns])
    
    X_test[continuous_columns] = scalers[location].transform(X_test[continuous_columns])

    return X_test, X_training

def make_observed_and_estimated_category(df_observed, df_estimated, df_test):
     # Hot encode in wether observed or estimated
    df_observed['observed_or_estimated'] = 'observed'
    df_estimated['observed_or_estimated'] = 'estimated'
    df_test['observed_or_estimated'] = 'estimated'
    # Concatenate observed and estimated
    df_training = pd.concat([df_observed, df_estimated], axis=0).sort_values(by="date_forecast")
    df_training['observed_or_estimated'] = df_training['observed_or_estimated'].astype('category')
    df_test['observed_or_estimated'] = df_test['observed_or_estimated'].astype('category')

    return df_training, df_test

def analyze_pv_measurements(dataframe):
    # Filter the DataFrame for rows where 'is_day' is 0
    night_data = dataframe[dataframe['is_day:idx'] == 0]

    # Analysis of 'pv_measurements' during night time
    pv_stats = night_data['pv_measurement'].describe()

    return pv_stats

    
    

In [16]:

# Initialize a dictionary to hold the scalers for each location

scalers = {}
scaling_target = False

def prepare_data(location):
    # Load data

    scaling = True  # Set scaling to True to enable individual scaling for each location
    global scalers
    global scaling_target
    # Load training data
    df_observed = pd.read_parquet(f"data/{location}/X_train_observed.parquet")
    df_estimated = pd.read_parquet(f"data/{location}/X_train_estimated.parquet")
    df_target = pd.read_parquet(f"data/{location}/train_targets.parquet")


    # Load test data
    df_test = pd.read_parquet(f"data/{location}/X_test_estimated.parquet")

    # Add calculated date
    df_observed, df_estimated, df_test = add_calc_date(df_observed, df_estimated, df_test)
    # Hot encode in wether observed or estimated

    # Resample and add custom fields

    df_observed = resample_add_data(df_observed, False)
    df_estimated = resample_add_data(df_estimated, False)
    df_test = resample_add_data(df_test, True)

    df_training, df_test = make_observed_and_estimated_category(df_observed, df_estimated, df_test)

    # Merge training with target data
    df_training = pd.merge(df_training, df_target, left_on="date_forecast", right_on="time", how="inner")
    
    # Create lagged features and remove outliers training
    df_training = create_weather_lagged_features(df_training, WEATHER_FEATURES)
    df_training = df_training[COLUMNS_TO_KEEP]
    df_training = remove_outliers(df_training)

    # Create lagged features test
    df_test = create_weather_lagged_features(df_test, WEATHER_FEATURES)
    df_test = df_test[TEST_COLUMNS_TO_KEEP]

    print(analyze_pv_measurements(df_training))
    # Make training x and y
    y_training = df_training["pv_measurement"]
    X_training = df_training.drop("pv_measurement", axis=1)

    # Add categories
    X_training = add_location_feature(X_training, location)
    X_test = add_location_feature(df_test, location)
    
    # Add scaling
    if scaling:
        X_test, X_training = add_scaling(X_test, X_training, location)
    X_test.reset_index(inplace=True)
    X_test.drop("date_forecast", axis=1, inplace=True)
    # y_training = np.log1p(y_training)
    if scaling_target:
        if location == "B": 
            y_training = A_B_ratio*(y_training)
        if location == "C":
            y_training = A_C_ratio*(y_training)
    return X_training, X_test, y_training



# Use prepare_data function

combined_X_train = pd.DataFrame()
combined_X_val = pd.DataFrame()
combined_Y_train = pd.DataFrame()
combined_Y_val = pd.DataFrame()
combined_X_test = pd.DataFrame()

# For validation locally 
for location in locations:
    # Prepare the training data
    X_training, X_test, y_training = prepare_data(location)
    
    # Split and concatenate the training data
    X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, test_size=0.0001, random_state=42)
    combined_X_train = pd.concat([combined_X_train, X_train])
    combined_X_val = pd.concat([combined_X_val, X_val])
    combined_Y_train = pd.concat([combined_Y_train, y_train])
    combined_Y_val = pd.concat([combined_Y_val, y_val])
    combined_X_test = pd.concat([combined_X_test, X_test])

    # Spl

combined_X_test
combined_X_train, combined_Y_train = shuffle(combined_X_train, combined_Y_train, random_state=42)
combined_X_val, combined_Y_val = shuffle(combined_X_val, combined_Y_val, random_state=42)

for location in locations: 
    prepare_data(location)

/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


count    15976.000000
mean         0.146713
std          3.232574
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        206.580000
Name: pv_measurement, dtype: float64


/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


count    14821.000000
mean         0.257302
std          6.118606
min         -0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        273.412500
Name: pv_measurement, dtype: float64


/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


...

/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


count    15976.000000
mean         0.146713
std          3.232574
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        206.580000
Name: pv_measurement, dtype: float64


/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


count    14821.000000
mean         0.257302
std          6.118606
min         -0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        273.412500
Name: pv_measurement, dtype: float64
count    13238.000000
mean         0.088243
std          2.489824
min         -0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        137.200000
Name: pv_measurement, dtype: float64


/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()
/var/tmp/ipykernel_101452/4272354047.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('1H').mean()


## Train

In [17]:
def train_model(X_train, y_train, X_val, y_val, location):
    cat_features = [index for index, col in enumerate(X_train.columns) if col in ['location', 'dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx', 'observed_or_estimated']]
    
    model = CatBoostRegressor(
        iterations=6000,
        learning_rate=0.007,
        depth=12,  # assuming you decided to keep the depth reduced
        loss_function='MAE',
        verbose=200,
        # per_float_feature_quantization='65:border_count=1024',
        cat_features=cat_features,
    )

    # Use the provided validation set for early stopping
    
    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=False,)
    model.save_model(f"catboost_model_merged.cbm")

def evaluate_model(X_val, y_val, location):
    global scaling_target  
    model = CatBoostRegressor()
    model.load_model(f"catboost_model_merged.cbm")
    
    # Unique locations in the X_val dataframe
    locations = X_val['location'].unique()
    
    for location in locations:
        # Filter X_val and y_val for the current location
        X_val_loc = X_val[X_val['location'] == location]
        y_val_loc = y_val[X_val['location'] == location]
        
        # Make predictions
        y_pred_loc = model.predict(X_val_loc)
        
        # Apply the transformation if needed (assuming y_val_loc and y_pred_loc are log1p transformed)
        # y_val_loc = np.expm1(y_val_loc)
        # y_pred_loc = np.expm1(y_pred_loc)
        
        # Calculate MAE for the current location
        mae = mean_absolute_error(y_val_loc, y_pred_loc)

        if scaling_target: 
            if location == "B": 
                mae = mae/A_B_ratio
            if location == "C":
                mae = mae/A_C_ratio
                
        print(f'Location {location}, Mean Absolute Error: {mae}')


# Train the model using all available training data and the initial validation set for early stopping
train_model(combined_X_train, combined_Y_train, combined_X_val, combined_Y_val, location)
# Evaluate the model using the same validation set
evaluate_model(combined_X_val, combined_Y_val, location)


0:	learn: 291.1747725	test: 748.0749860	best: 748.0749860 (0)	total: 264ms	remaining: 26m 24s
200:	learn: 120.4355877	test: 340.3530055	best: 340.3530055 (200)	total: 39.1s	remaining: 18m 47s
400:	learn: 87.9791543	test: 258.5880338	best: 258.4648335 (399)	total: 1m 17s	remaining: 18m 2s
600:	learn: 77.9862214	test: 231.5580906	best: 231.5580906 (600)	total: 1m 56s	remaining: 17m 23s
800:	learn: 73.9952100	test: 219.2543227	best: 219.2543227 (800)	total: 2m 34s	remaining: 16m 42s
1000:	learn: 70.9904915	test: 214.3781165	best: 214.3781165 (1000)	total: 3m 13s	remaining: 16m 7s
1200:	learn: 69.7845306	test: 214.7317033	best: 214.0832978 (1128)	total: 3m 52s	remaining: 15m 30s
1400:	learn: 68.8523475	test: 214.5354891	best: 214.0832978 (1128)	total: 4m 31s	remaining: 14m 50s
1600:	learn: 67.5184148	test: 213.3025232	best: 213.2961239 (1598)	total: 5m 10s	remaining: 14m 11s
1800:	learn: 65.9367814	test: 212.6226062	best: 212.5849724 (1799)	total: 5m 49s	remaining: 13m 34s
2000:	learn: 64.

## Print feature importance

In [18]:
def feature_importance():
    model = CatBoostRegressor()
    model.load_model(f"catboost_model_merged.cbm")

    # Getting feature importances
    feature_importances = model.get_feature_importance()
    feature_names = model.feature_names_

    # Creating a DataFrame from feature importances
    df_feature_importances = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    })

    # Sorting the DataFrame by importance in descending order
    df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=False)

    return df_feature_importances


# feature_importances = feature_importance()
# Calling the function
# print(feature_importances.head(25))

## Make predictions

In [19]:

# Assuming you have defined WEATHER_FEATURES, TEST_COLUMNS_TO_KEEP, and other functions previously

def make_predictions(df_test_pred):
    model = CatBoostRegressor()
    model.load_model(f"catboost_model_merged.cbm")
    # Load model 
    model = CatBoostRegressor()
    model.load_model(f"catboost_model_merged.cbm")
    
    preds = model.predict(df_test_pred)
    
    # Inverse transform the predictions
    # preds = np.expm1(preds)
    return preds





## Evaluate locally


In [20]:
import matplotlib.pyplot as plt



def evaluate_model_locally(location, X_val_loc, y_val_loc):
    # Load the test data
    global scaling_target
    
    # Make predictions
    preds = make_predictions(X_val_loc)[-200:]
    target = y_val_loc.values[-200:]
    if scaling_target:
        if location == "B": 
            preds = (preds)/A_B_ratio
        
        if location == "C":
            preds = (preds)/A_C_ratio

    # differences = preds - target

    # # Count predictions lower than the actual
    # lower_predictions = (differences < 0) & (target != 0)
    # # Count predictions higher than the actual
    # higher_predictions = (differences > 0) & (target != 0)

    # # Biggest misreads
    # absolute_differences = abs(differences)
    # max_diff_index = absolute_differences.argmax()  # Index of the biggest difference
    # max_diff_value = absolute_differences[max_diff_index]  # Value of the biggest difference
    # print(f"Number of predictions that are a lower value than the actual, given that the actual is not 0: {lower_predictions.sum()}")
    # print(f"Number of predictions that are larger than the target, given that the target is not 0: {higher_predictions.sum()}")
    # print(f"The biggest misread is at index {max_diff_index} with a difference of {max_diff_value}")

    # write preds to a csv file
    # df = pd.DataFrame(preds)
    # df.to_csv(f"best_preds_{location}.csv")
    # make array of indices of numpy array target 

    # plot the preds from the csv file "best_preds_{location}.csv"
    df = pd.read_csv(f"best_preds_{location}.csv")[-200:]
    # df.reset_index(inplace=True)
    best_preds = df["0"]
    print(preds)
    index = np.arange(len(target))
    # Plotting
    plt.figure(figsize=(60,6))
    plt.plot(index, target, label="Target")
    plt.plot(index, preds, label="Predictions")
    plt.plot(index, best_preds.values, label="Best predictions")

    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Target vs Predictions')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()

for location in locations:
    X_val_loc = combined_X_val[combined_X_val['location'] == location].sort_index()
    y_val_loc = combined_Y_val[combined_X_val['location'] == location].sort_index()
    # evaluate_model_locally(location, X_val_loc, y_val_loc)


## Submit to csv 1

In [21]:
df_submission = pd.read_csv("data/test.csv")

preds = make_predictions(combined_X_test)
print(len(preds))
print(len(df_submission))
df_submission["prediction"] = preds

# Save the results to a new submission file
df_submission[["id", "prediction"]].to_csv("Short-notebook-Catboost-145.csv", index=False)

2160
2160


# Ensamble average Autogluon and Catboost

In [26]:
# Load the datasets
df1 = pd.read_csv("Short-notebook-Catboost-145.csv")
df2 = pd.read_csv("Short-notebook-Autogluon-erik.csv")
# Verify that the IDs match in all DataFrames (assuming they are sorted and have the same length)
# if not (df1['id'].equals(df2['id']) and df1['id'].equals(df3['id']) and df1['id']):
#     raise ValueError("The IDs in the CSV files do not match.")

# Calculate the average of the predictions for all rows
df_average = pd.DataFrame({
    'id': df1['id'],
    'prediction': (df1['prediction'] + df2['prediction'])/2
})

df = df_average

# Ensure there are no negative predictions
df['prediction'] = df['prediction'].apply(lambda x: max(x, 0))

df['prediction'] = df['prediction'].apply(lambda x: 0 if x < 1.5 else x)


# Set predictions to zero where up to three non-zero predictions are surrounded by zeros
for i in range(1, len(df) - 1):
    # Check single non-zero prediction surrounded by zeros
    if df.loc[i - 1, 'prediction'] == 0 and df.loc[i + 1, 'prediction'] == 0:
        df.loc[i, 'prediction'] = 0
    # Check two consecutive non-zero predictions surrounded by zeros
    if i < len(df) - 2 and df.loc[i - 1, 'prediction'] == 0 and df.loc[i + 2, 'prediction'] == 0:
        df.loc[i, 'prediction'] = 0
        df.loc[i + 1, 'prediction'] = 0
    # Check three consecutive non-zero predictions surrounded by zeros

# Save the averaged predictions to a new CSV file
df.to_csv('predictions/short_notebook_cat_autogluon.csv', index=False)